# Chap03 - 신경망 시작하기

## 핵심 내용

- 2장에서 살펴본 층(layer), 네트워크(network), 목적 함수, 옵티마이저와 같은 신경망의 핵심 구성요소들에 대해 살펴본다.
- 파이썬 딥러닝 라이브러리인 케라스(Keras)에 대해 간략하게 알아본다.
- 텐서플로, 케라스, GPU를 사용한 딥러닝을 실행하기 위해 컴퓨터를 셋팅한다.
- 실전 문제를 해결하기 위해 신경망을 어떻게 사용하는지 세 가지 기본 예제로 알아본다.
  - 영화 리뷰를 긍정 또는 부정으로 분류하기 (이진 분류, binary classification)
  - 신문 기사를 토픽으로 분류하기 (다중 분류, multiclass classification)
  - 부동산 데이터를 바탕으로 주택 가격을 예측하기 (회귀, regression)

## 3.1 신경망의 구조

신경망의 학습에는 다음 요소들이 관련되어 있다.

- **네트워크**(또는 **모델**)을 구성하는 **층(레이어, layer)**
- **입력 데이터** 와 이에 대응되는 **타겟**(또는 레이블)
- 학습에 대한 평가 지표인 **손실함수**(loss function)
- 가중치 파라미터를 업데이트를 결정하는 방법인 **옵티마이저**



![](./images/network.PNG)


## 3.1.1 층: 딥러닝의 구성 단위

- **층(레이어)**은 하나 이상의 텐서를 입력으로 받아 하나 이상의 텐서를 출력하는 데이터 처리 모듈이다.
- 어떤 종류의 층은 상태(state)가 없지만 대부분의 경우 **가중치**(weight)라는 층의 상태를 가진다.
  - 플랫튼(Flatten), 풀링(Pooling), 드롭아웃(Dropout) 등과 같은 층에는 학습되는 가중치가 없다.
  - 보통 신경망의 가중치를 상태라고 하지 않지만,6장에서 소개될 순환 신경망(RNN) 셀(Cell)의 출력은 상태(hidden state, $h_t$)라고 한다.
- 가중치는 경사 하강법(Gradient Descent)에 의해 학습되는 하나 이상의 텐서이다.
- 층마다 적절한 텐서 포맷과 데이터 처리 방식이 다르다.
  - **완전 연결(fully connected) 층** : dense layer라고도 하며, `(samples, features)` 크기의 2D 텐서를 받는다.
  - **순환 층(recurrent layer)** : `(samples, timesteps, features)` 크기의 3D 텐서의 시퀀스 데이터를 받으며, RNN에서 사용된다.
  - **합성곱 층(convolution layer)** : `(samples, height, width, channels)` 크기의 4D 텐서의 이미지 데이터를 받는다.
  

#### 케라스의 층 호환성(layer compatibility)

- 케라스(keras)는 호환 가능한 층들을 엮어 데이터 변환 파이프라인(pipeline)을 구성함으로써 딥러닝 모델을 만든다.
- 케라스는 층 호환성(layer compatibility)를 통해 각 층이 특정 크기의 입력 텐서만 받고 특정 크기의 출력 텐서를 반환하는 것을 자동으로 구성해준다.
- 아래의 코드에서 처럼 두 번째 층에는 `input_shape` 인자를 지정하지 않아도 층 호환성으로 인해 자동으로 앞의 층의 입력 크기를 받는다. 

```python
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784, )))  # Input: [None, 784]
model.add(layers.Dense(10))  # Input: [None, 32]
model.summary()```

- 두번째 층에는 앞선 층의 출력 크기를 입력 크기로 자동으로 채택한다.

### 3.1.2 모델: 층의 네트워크

- 딥러닝 모델은 층(layer)으로 만든 비순환 유향 그래프(Directed Acyclic Graph, DAG)이다.
- 네트워크 구조는 **가설 공간**(hypothesis space)을 정의한다. 
- 네트워크 구조를 선택함으로써 **가능성 있는 공간**(가설 공간)을 입력 데이터에서 출력데이터로 매핑하는 일련의 특정 텐서연산으로 제한한다.

### 3.1.3 손실함수와 옵티마이저

- **손실함수**(loss function) : 훈련하는 동안 최소화될 값이다. 주어진 문제에 대한 성공 지표가 된다.
  - 회귀(regression) 문제일 때는 대표적으로 **평균제곱오차**(MSE, Mean Squared Error)을 사용한다.
  - 분류(classification)문제일 때는 대표적으로 **교차 엔트로피 오차**(CEE, Cross Entropy Error)를 사용한다.
- **옵티마이저**(optimizer) : 손실 함수를 기반으로 네트워크의 가중치 파라미터가 어떻게 업데이트 될지 결정하는 방법이며, 특정 종류의 확률적 경사 하강법(SGD)를 구현한다.

## 3.2 케라스(Keras) 소개

[케라스](https://keras.io)의 특징은 다음과 같다.

- 동일한 코드로 CPU와 GPU에서 실행할 수 있다.
- 사용하기 쉬운 API를 가지고 있어 딥러닝 모델의 포로토타입을 빠르게 만들 수 있다.
- 다양한 층(layer)을 지원하며 서로 자유롭게 조합하여 사용할 수 있다.
- 다중 입력, 다중 출력, 층의 공유, 모델 공유 등 다양한 네트워크 구조를 만들 수 있다.

- 케라스는 MIT 라이선스를 따르므로 상업적인 프로젝트에도 자유롭게 사용할 수 있다.

### 3.2.1 케라스, 텐서플로, 씨아노, CNTK

- 케라스는 딥러닝 모델을 만들기 위한 고수준의 구성 요소를 제공하는 라이브러리이다.
- 텐서 조작이나 미분 같은 low-level은 케라스의 **벡엔드 엔진**(TensorFlow, Theano 등)을 사용한다.

![](./images/keras.PNG)

- 케라스로 작성한 모든 코드는 아무런 변경없이 위의 백엔드 중 하나를 선택해 실행할 수 있다.
- 케라스는 텐서플로를 기본 백엔드로 사용되며, CPU와 GPU에서 모두 작동한다.
  - CPU일 경우에는 텐서플로는 [Eigen](http://eigen.tuxfamily.org)이라는 low-level의 텐서 연산 라이브러리를 이용한다.
  - GPU일 경우에는 NVIDIA의 cuDNN이라는 딥러닝 연산 라이브러리를 사용한다.